In [2]:
%run "../../PyUtils/common.ipynb"

from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

#han1="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20patient_data_table";
#hand="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20hand"
#kimj="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20kimj";
#leeh="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20leeh";
#sony="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20sony";
#kuky="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20kuky";


patient = "kimj"
ampm = "am"
file = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20calculated_jlee%20WHERE%20ampm=%27"+ampm+"%27%20AND%20pname=%27"+patient+"%27%20ORDER%20BY%20date,%20time"

fileName = file;

dfOriginal = LoadDataSet(fileName, checkForDateTime=False);
display (dfOriginal )
#for c in dfOriginal.columns: print (c,  end=', ')

<IPython.core.display.Javascript object>

,pname,date,ampm,time,pef,pef_zone,tm_window,so2_max,co_max,o3_max,no2_max,pm10_max,so2_sum,co_sum,o3_sum,no2_sum,pm10_sum,so2_avg,co_avg,o3_avg,no2_avg,pm10_avg,temperaturec,windspeedms,winddirection,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperature_max,airpressure_max,precipitation_max,temperature_min,airpressure_min,precipitation_min,temperature_diff,airpressure_diff,precipitation_diff
0,kimj,2015-02-06,am,06:09:00,85,R,24.367,0.018,1.100,0.041,0.057,125.000,0.237,19.578,0.605,0.920,2115.567,0.010,0.803,0.025,0.038,86.822,-4.700,0.500,110.000,93.000,4.000,-5.600,1021.900,1026.300,-2.800,4.700,1021.900,93.000,-4.700,1018.000,42.000,9.400,3.900,51.000
1,kimj,2015-02-07,am,06:34:00,123,R,24.417,0.012,1.300,0.033,0.069,81.000,0.175,18.618,0.267,1.215,1262.033,0.007,0.763,0.011,0.050,51.687,-1.900,0.800,200.000,91.000,4.900,-3.100,1018.900,1023.300,-1.300,6.100,1023.000,94.000,-4.700,1018.900,44.000,10.800,4.100,50.000
2,kimj,2015-02-08,am,09:26:00,128,R,26.867,0.012,1.300,0.040,0.072,175.000,0.171,13.707,0.712,0.599,2000.300,0.006,0.510,0.026,0.022,74.453,-9.800,4.600,320.000,33.000,1.000,-23.000,1022.700,1027.200,-4.100,5.100,1022.700,91.000,-10.400,1015.500,32.000,15.500,7.200,59.000
3,kimj,2015-02-09,am,06:13:00,97,R,20.783,0.009,0.400,0.033,0.015,166.000,0.091,6.657,0.563,0.144,1573.650,0.004,0.320,0.027,0.007,75.717,-11.300,1.000,290.000,45.000,13.042,-20.900,1026.600,1031.200,-6.100,-5.700,1026.600,46.000,-11.300,1022.300,29.000,5.600,4.300,17.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,kimj,2016-02-01,am,09:36:00,126,R,23.767,0.007,0.600,0.035,0.031,51.000,0.115,10.743,0.664,0.385,953.000,0.005,0.452,0.028,0.016,40.098,-5.500,1.500,320.000,42.000,1.700,-16.400,1026.300,1030.800,-6.300,0.400,1026.500,58.000,-7.900,1024.400,31.000,8.300,2.100,27.000
361,kimj,2016-02-02,am,08:36:00,97,R,23.000,0.010,1.200,0.032,0.064,69.000,0.134,12.760,0.495,0.603,1091.400,0.006,0.555,0.022,0.026,47.452,-9.900,0.300,0.000,64.000,1.800,-15.400,1025.800,1030.300,-8.900,-1.000,1027.000,64.000,-9.900,1024.200,25.000,8.900,2.800,39.000
362,kimj,2016-02-03,am,08:57:00,158,R,24.350,0.011,1.400,0.036,0.073,87.000,0.171,20.610,0.317,1.127,1378.250,0.007,0.846,0.013,0.046,56.602,-8.400,0.000,0.000,76.000,2.500,-11.800,1024.900,1029.400,-7.700,1.200,1026.100,76.000,-9.900,1022.500,24.000,11.100,3.600,52.000
363,kimj,2016-02-04,am,05:47:00,98,R,20.833,0.011,1.500,0.043,0.073,103.000,0.136,19.167,0.353,0.937,1295.867,0.007,0.920,0.017,0.045,62.202,-3.400,1.100,180.000,86.000,4.100,-5.400,1022.400,1026.800,-3.900,4.700,1025.300,87.000,-8.400,1022.400,35.000,13.100,2.900,52.000


In [3]:
df=dfOriginal.copy()
df.insert(4, 'npt',0)
df.insert(4, 'pef',0)
df.npt = pd.notnull(df.pef1)*1 + pd.notnull(df.pef2)*1 + pd.notnull(df.pef3)*1
df = df[df.npt > 0]
df=df.reset_index(drop=True)
df.pef = (df.pef1 + df.pef2 + df.pef3)/df.npt

df=df.fillna(0)
df.loc[df.timeofday == 'null', 'timeofday'] = "00:00:00"
sdttm = df.dateofmeasure + " " + df.timeofday 
df.insert(0, 'sdttm', sdttm)
df.sdttm = pd.to_datetime(df.sdttm)

df.sort_values(by='sdttm', ascending=True, inplace=True)
drps  = "cname, ampm, timeofday, dateofmeasure, npt, pef1, pef2, pef3, pef, indexpef".split(', ')
df=df.drop(drps, axis=1, errors='ignore')
df=df.reset_index(drop=True)

## <== do the following for SAP PA tool
#pef = df.pef;
#df=df.drop(['pef'], axis=1)
#df.insert(1, 'pef1', pef)

#pef[-10:]=0     # Set last 10 to zero for predictions
#df.insert(1, 'pef', pef)

#df.to_csv("HanD/hand1.csv", sep=';')
df1 = df.copy()
df1 = df1.set_index(df1.sdttm)
df1=df1.drop('sdttm', axis=1, errors='ignore')
df1

ValueError: cannot insert pef, already exists

In [ ]:
for c in df1.columns:
    print(c, end=', ')

scaler = StandardScaler()
dp = prepareDF(df1, True)
for c in dp.columns:
    dp[c] = scaler.fit_transform(dp[c])

plt.figure(figsize=(25, 8))
plt.rc('font', size=14)

#dp = dp[0:90]
#dp = dp["pef,so2,co,o3,no2,pm10,tmax,so2poi,copoi,o3poi,no2poi,pm10poi,temperaturec".split(',')]
#dp = dp["pef,so2,co,o3,no2,pm10,temperaturec,rainfallmm,windspeedms,vaporpressurehpa".split(',')]
#dp = dp["pef,so2poi,copoi,o3poi,no2poi,pm10poi,temperaturec,rainfallmm,windspeedms,vaporpressurehpa".split(',')]

rolmean = pd.rolling_mean(dp.pefmax, window=14)

plt.plot(dp.index, dp.pefmax, color='b', linewidth=3)
#plt.plot(dp.index, rolmean, color='r', linewidth=2, label='trend')
#plt.plot(dp.index, dp.so2)
plt.plot(dp.index, dp.co)
#plt.plot(dp.index, dp.o3)
plt.plot(dp.index, dp.no2)
#plt.plot(dp.index, dp.pm10)
#plt.plot(dp.index, dp.tmax)
#plt.plot(dp.index, dp.amax)
#plt.plot(dp.index, dp.pmax)

plt.legend()

In [ ]:
#display(HTML(dp.to_html()) )


In [ ]:
c="r,g,b,c,m,y,k,w".split(",")
dpc = dp["pefmax,so2,co,o3,no2,pm10,tmax,so2poi,copoi,o3poi,no2poi,pm10poi,temperaturec".split(',')]
axes = scatter_matrix(dpc, alpha=0.2, diagonal='kde', figsize=(13, 10)); # diagonal can be 'hist';

corr = dpc.corr().as_matrix()
for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.3f" %corr[i,j], (0.8, 0.8), xycoords='axes fraction', ha='center', va='center')
plt.show()


In [ ]:
def run_reg(X,y,reg_class,printDebug = False , reg=None):
    # Construct a kfolds object
    kf = sklearn.model_selection.KFold(n_splits=5, shuffle=True)
    y_pred = y.copy()

    # Iterate through folds\
    i = 0;
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = reg_class(**kwargs) if (reg is None)  else reg;
        if (printDebug): print ("*",i, end ="");
        reg.fit(X_train,y_train)
        y_pred[test_index] = reg.predict(X_test)
        i = i +1;
    if (printDebug): print ("*", end="");
        
    m = mean_squared_error(y, y_pred)

    return y_pred, m, clf

In [ ]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcess
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from statsmodels.regression.quantile_regression import QuantReg
import statsmodels.formula.api as smf

params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.05, 'loss': 'quantile', 'alpha':.95}


clfs = [
    ensemble.GradientBoostingRegressor(**params),
    ensemble.GradientBoostingRegressor(loss='quantile', alpha=.95,
                                n_estimators=250, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9),
    DecisionTreeRegressor(max_depth=2), 
    DecisionTreeRegressor(max_depth=15),
    linear_model.LinearRegression(),
    RandomForestRegressor(max_depth=50, random_state=0),
    AdaBoostRegressor(),
#    Lasso(alpha=0.2,normalize=True, max_iter=1e5),
#    Lasso(alpha=0.5,normalize=True, max_iter=1e5),
    Ridge(alpha=0.1,normalize=True),
    Ridge(alpha=0.5,normalize=True),

#    GaussianProcess(corr='squared_exponential', theta0=1e-1, thetaL=1e-3, thetaU=1,random_start=100),
#    GaussianProcess( theta0=1e-1, thetaL=1e-3, thetaU=1,random_start=100),

    SVR(kernel='rbf', C=1e3, gamma=0.1),
    SVR(kernel='linear', C=1e3),
    SVR(kernel='poly', C=1e3, degree=2)  ,
]

#rclfs={}


In [ ]:
#dp = dp["so2,pef,o3,co,no2,pm10,tmax".split(',')]

dp1 = dp.set_index(pd.Series(range(0,len(dp)) ) )

X = dp1.ix[:, dp1.columns != 'pefmax']
y = dp1.pefmax
X = X.as_matrix().astype(np.float)

##---- Polynomial features
poly=2
p = PolynomialFeatures(poly)
o = p.fit_transform(X)
X = o


plt.figure(figsize=(25, 8))
h1, h2 =-100, None
h1, h2 =500, 550
plt.plot(dp.index[h1:h2], y[h1:h2], label='original PEF', linewidth=3)


for i, r in enumerate(clfs):
    d1 = datetime.datetime.now()
    print ("+Run Start {:2} {} : {}".format(i, d1, str(r)[0:25]) , end=' ')
    
    if ( rclfs.get(str(r)) is not None ):
        print ("{} already exists:".format(str(r)[0:15]))
        continue;
        
    yp, mse,clf = run_reg(X,y, None, True, r)
    
    d2 = datetime.datetime.now()
    dur = d2 -d1;
    dur = dur - datetime.timedelta(microseconds=dur.microseconds)
    rclfs[str(clf)]=(yp, mse, clf, dur ) 
    l='{} MSE {:04.2} {}'.format( str(clf)[0:7]+'..'+str(c[-5]), mse, str(dur))
    print('-', l)
    
    plt.plot(dp.index[h1:h2], yp[h1:h2], label=l )
    #if ( i > 11):
    #    break;


plt.legend()


In [ ]:
def plotcl(rclfs, clf, dp):
    poly=1
    yp, mse, clf, df = rclfs[clf]
    l='[ {} ] : MSE {:05.2} {} {}'.format( str(clf)[0:17]+'..'+str(c[-10]), mse, str(df), poly)
    plt.figure(figsize=(25, 5))
    h1, h2 =-100, None
#    h1, h2 =500, 550
    plt.plot(dp.index[h1:h2], y[h1:h2], label='original PEF', linewidth=3)
    plt.plot(dp.index[h1:h2], yp[h1:h2], label=l )
    plt.legend()
    print(l)
        
for clf in rclfs.keys():
    plotcl(rclfs, clf, dp)
#    break;